# Library

In [1]:
# 標準ライブラリのインポート
import os
import json

# サードパーティライブラリのインポート
import requests
from dotenv import load_dotenv
import pandas_datareader.data as web 
import xml.etree.ElementTree as ET

# LLM関連のインポート
from openai import OpenAI

# APIキーの設定
load_dotenv() 
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# GPTモデルの設定
model_name='gpt-4o-mini-2024-07-18' #'gpt-4o-2024-08-06'

# 株価を取得する関数

In [2]:
# DataReaderでstooqから株価情報を取得する関数を定義（開始日～終了日）
def get_stock_price(code, date_from ,date_to):
    # 日本用のコードに変換
    code = code + '.JP'
    # stooqからデータ取得
    df = web.DataReader(code, data_source='stooq', start=date_from,end=date_to)
    # 日付と株価の終値を返す ※日付形式がおかしくなるのでorient='table'を利用
    return df['Close'].to_json(orient='table')

def get_stock_price_range(arguments):
    res = get_stock_price(
        code = arguments.get('code'),
        date_from = arguments.get('date_from'),
        date_to = arguments.get('date_to')
    )
    return res

# 天気を取得する関数

In [3]:
# 天気情報を取得する関数を定義
def get_info(latitude, longitude):
    url = "https://api.open-meteo.com/v1/forecast"
    parameters = {
        "latitude": latitude,
        "longitude": longitude,
        "current_weather": "true"}
    response = requests.get(url, params=parameters)

    if response.status_code == 200:
        data = response.json()
        return json.dumps(data["current_weather"])
    else:
        return None

def get_weather_by_location(arguments):
    res = get_info(
        # 緯度と経度の情報 取得
        latitude = arguments.get('latitude'),
        longitude = arguments.get('longitude')
    )
    return res

# 書籍情報を取得する関数

In [4]:
# 書籍情報を取得する関数を定義
def get_books(keyword):
    url = "https://ci.nii.ac.jp/books/opensearch/search"
    parameters = {
        "q": keyword
    }
    response = requests.get(url, params=parameters)

    if response.status_code == 200:
        try:
            # Parse the XML response
            root = ET.fromstring(response.content)
            
            # Extract book titles and other relevant information
            books = []
            for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
                title = entry.find('{http://www.w3.org/2005/Atom}title').text
                link = entry.find('{http://www.w3.org/2005/Atom}link').attrib['href']
                author = entry.find('{http://purl.org/dc/elements/1.1/}publisher').text if entry.find('{http://purl.org/dc/elements/1.1/}publisher') is not None else 'Unknown'
                
                books.append({
                    "title": title,
                    "link": link,
                    "author": author
                })
            
            return json.dumps(books, ensure_ascii=False)  # Ensure correct encoding for Japanese characters
        except ET.ParseError as e:
            st.error("Failed to parse XML. Error:")
            st.write(str(e))
            return None
    else:
        st.error(f"Error: {response.status_code}")
        st.write(response.text)
        return None

def get_books_by_keyword(arguments):
    res = get_books(
        keyword=arguments.get('keyword')
    )
    return res

# Function callingの設定

In [5]:
# 使用するツールのリスト
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_price_range",  
            "description": "証券コードと2つの日付を渡すと指定の会社のその2つの日付の間の株価の終値を返します",  
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "証券コード",
                    },
                    "date_from": {
                        "type": "string",
                        "description": "株価を知りたい日付の開始日",
                    },
                    "date_to": {
                        "type": "string",
                        "description": "株価を知りたい日付の終了日",
                    },
                },
                "required": ["code", "date_from", "date_to"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather_by_location",  
            "description": "緯度と経度の情報から現在と将来の天気を取得",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {
                        "type": "string",
                        "description": "緯度",
                    },
                    "longitude": {
                        "type": "string",
                        "description": "経度",
                    },
                },
                "required": ["latitude", "longitude"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_books_by_keyword",
            "description": "キーワードで書籍情報を検索し、関連する書籍情報を取得します",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "書籍情報を取得するための検索キーワード",
                    },
                },
                "required": ["keyword"],
            },
        },
    },
]

# LLM

In [6]:
def llm_agent(user_input):

    client = OpenAI()

    messages = [
        {"role": "system", "content": "You are a helpful assistant. Use the supplied tools to assist the user if needed."},
        {"role": "user", "content": user_input}
    ]

    # ChatGPT API呼び出し
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        tools=tools,
        tool_choice='auto',
    )

    # ツール呼び出しの結果があるか確認
    tool_calls = getattr(response.choices[0].message, 'tool_calls', None)  # tool_callsが存在しない場合はNone

    if tool_calls:
        for tool_call in tool_calls:
            function_name = tool_call.function.name  # 属性としてアクセス
            arguments = json.loads(tool_call.function.arguments)

            # 関数の種類に基づいて処理を呼び出す
            if function_name == 'get_stock_price_range':
                function_response = get_stock_price_range(arguments)
            elif function_name == 'get_weather_by_location':
                function_response = get_weather_by_location(arguments)
            elif function_name == 'get_books_by_keyword':
                function_response = get_books_by_keyword(arguments)
            else:
                raise NotImplementedError(f"Unknown function: {function_name}")

            # ツール呼び出しごとに適切に応答を作成
            second_response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {'role': 'system', 'content': "You are a helpful assistant."},
                    {"role": "user", "content": user_input},
                    {"role": "assistant", "tool_call_id": tool_call.id, "content": ""},  # 空文字列を設定
                    {
                        'role': 'function',
                        'name': function_name,
                        'content': str(function_response),
                    },
                ]
            )
            # 最終的な応答を表示
            print(second_response.choices[0].message.content)
    else:
        # 関数が呼び出されていない場合、通常の応答を表示
        print(response.choices[0].message.content)


In [7]:
llm_agent('三井住友フィナンシャルグループの2024年8月の株価の動きと、9月の見通しについて分析してください。')

三井住友フィナンシャルグループの2024年8月の株価は、月初から急激に変動しました。以下にその動きを分析します。

### 2024年8月の株価の動き
- **月初の株価**: 8月1日の株価は10,880円でした。
- **中旬の下落**: 8月2日の株価は9,662円ですが、その後金額は急激に下がり、8月6日には8,000円まで落ち込みました。
- **その後の回復**: 8月9日には8,968円、8月14日には9,397円と徐々に回復していきました。
- **高値と低値の変動**: 最も高い株価は8月1日の10,880円であり、最も低い株価は8月6日の8,000円という大きな幅が見られました。
- **月末の株価**: 8月30日の株価は9,550円で、上昇傾向にあることを示していますが、8月23日には9,889円に達したのち、再度価格が下がる動きが見られました。

### 9月の見通し
株価の変動を考慮すると、9月には以下の要因が影響する可能性があります。

1. **経済指標の影響**: 日本国内や世界経済の動向に依存するものの、特に金融関連のニュースや経済指標（GDP、失業率、インフレ率など）に敏感に反応する可能性があります。
2. **市場心理**: 8月の上昇と下落が混在していることから、市場参加者の心理が冷静さを欠くことも考えられます。リスク回避的な姿勢が強まると、売りが優先される可能性があります。
3. **技術的なサポートラインとレジスタンス**: 9,000円から10,000円のレンジが今後の取引の重要な指標となります。この範囲の中で動きが続く場合、突破に向けたシグナルを投資家が探るでしょう。
4. **外部要因**: 米国の金利政策や国際的な地政学的リスクも株価に影響を与える可能性があります。

### 結論
三井住友フィナンシャルグループの株価は、8月を通じてかなりのボラティリティが見られます。9月に入ると、経済指標の動向や市場心理に基づいてさらなる変動が予測されます。短期的には保守的な立場を取ることが重要であり、十分な情報収集を行いながら投資判断を行うことが推奨されます。
三井住友フィナンシャルグループの2024年8月から9月の株価の動きについて分析します。

### 2024年8月の株価動向
2024年8月の株価の詳細は示されていませ

In [8]:
llm_agent('東京の天気を教えてください。')

現在の東京の天気は、以下の通りです：

- **気温**: 25.0°C
- **風速**: 2.2 m/s
- **風向**: 東 (90°)

昼間の時間帯ではなく、夜の時間帯です。晴れています。必要な情報がありましたら、お知らせください。


In [9]:
llm_agent('生成AIに関する最新の書籍を教えてください。')

以下は、生成AIに関する最新の書籍のリストです。興味のある書籍をクリックすると、詳細情報を確認できます。

1. [生成AIの論点 : 学問・ビジネスからカルチャーまで](https://ci.nii.ac.jp/ncid/BD07157185) - 著者: 青弓社
2. [生成AIパスポートテキスト&問題集 : GUGA公認](https://ci.nii.ac.jp/ncid/BD05525696) - 著者: 日本能率協会マネジメントセンター
3. [生成AI法務・ガバナンス : 未来を形作る規範](https://ci.nii.ac.jp/ncid/BD07709099) - 著者: 商事法務
4. [おとなもこどもも知りたい生成AIの教室](https://ci.nii.ac.jp/ncid/BD07706170) - 著者: カンゼン
5. [ChatGPT&画像生成AI](https://ci.nii.ac.jp/ncid/BD03282753) - 著者: インプレス
6. [Gemini完全マニュアル : Googleの対話型生成AI](https://ci.nii.ac.jp/ncid/BD07644528) - 著者: 秀和システム
7. [つくりながら学ぶ!生成AIアプリ&エージェント開発入門](https://ci.nii.ac.jp/ncid/BD07898730) - 著者: マイナビ出版
8. [AIで好きな絵をつくる!Stable Diffusion : 魔法の言葉、呪文(プロンプト)が満載!「画像生成AI」超入門](https://ci.nii.ac.jp/ncid/BD03703272) - 著者: 興陽館
9. [60分でわかる!生成AIビジネス活用最前線](https://ci.nii.ac.jp/ncid/BD06441660) - 著者: 技術評論社
10. [Open code interpreter自然言語生成AIコード開発](https://ci.nii.ac.jp/ncid/BD06990647) - 著者: [中山茂]
11. [生成AI : 社会を激変させるAIの創造力 = Generative AI](https://ci.nii.ac.jp/ncid/BD02336755) - 著者: SBクリエイティブ

In [10]:
llm_agent('LLMについて説明して下さい。')

LLM（Large Language Model）は、大規模なテキストデータを用いて訓練された言語モデルです。これらのモデルは、自然言語処理（NLP）タスクを実行するために設計されており、人間の言語を理解し生成する能力を持っています。

### 主な特徴

1. **大規模なデータセット**: LLMは、インターネット上の文章、書籍、ニュース記事など、さまざまなソースから収集した膨大なテキストデータを使用して訓練されます。

2. **自己教師あり学習**: トレーニング中、モデルはテキストの一部を隠し、何が隠されているかを予測するように訓練されます。この方法により、文脈を理解する能力が高まり、自然な言語生成が可能になります。

3. **多様なタスク**: LLMは、文章生成、質問応答、要約、翻訳など、さまざまな自然言語処理のタスクに利用できます。

4. **コンテキスト理解**: LLMは、与えられたテキストの文脈を理解し、それに基づいて適切な応答や情報を生成する能力を備えています。

5. **ファインチューニング**: プリトレーニングされたモデルは、特定のタスクやドメインに合わせてファインチューニングされることがあります。これにより、特定のニーズに応じたパフォーマンスが向上します。

### 使用例

- **チャットボット**: カスタマーサポートや情報提供のために使用される。
- **コンテンツ生成**: 記事やブログの自動作成に利用される。
- **教育**: 質問応答や学習教材の作成に役立つ。

### 代表的なモデル

- **GPTシリーズ**: OpenAIが開発したGenerative Pre-trained Transformerのシリーズ。
- **BERT**: Googleが開発したBidirectional Encoder Representations from Transformers。
- **T5**: Text-to-Text Transfer Transformerとして知られるモデル。

LLMはその能力の高さから、多くの分野で応用されており、今後も進化が期待されています。
